# Club Churn Analysis - Exploratory Data Analysis

This notebook performs exploratory data analysis on the club churn dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
import os

# Add the parent directory to sys.path to import config
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import config

In [2]:
# Load the training data using the path from config
df_train = pd.read_excel(config.TRAIN_DATA_PATH)

# Display the first few rows of the dataset
print(f"Dataset shape: {df_train.shape}")
df_train.head()

Dataset shape: (9325, 16)


,ID,MEMBERSHIP_NUMBER,MEMBERSHIP_TERM_YEARS,ANNUAL_FEES,MEMBER_MARITAL_STATUS,MEMBER_GENDER,MEMBER_ANNUAL_INCOME,MEMBER_OCCUPATION_CD,MEMBERSHIP_PACKAGE,MEMBER_AGE_AT_ISSUE,ADDITIONAL_MEMBERS,PAYMENT_MODE,AGENT_CODE,START_DATE,END_DATE,MEMBERSHIP_STATUS
0,5944,A05948,12,100000.00,NaN,M,399996.0,1.0,TYPE-B,43,2,ANNUAL,1194400,20110929,20121119.0,CANCELLED
1,5487,A05490,17,100514.23,M,F,549996.0,1.0,TYPE-A,40,2,ANNUAL,739244,20110311,20120410.0,CANCELLED
2,9089,A09094,12,100000.80,M,M,1299996.0,1.0,TYPE-B,40,2,MONTHLY,1341388,20130625,NaN,INFORCE
3,5338,A05341,12,100000.99,M,M,500004.0,1.0,TYPE-B,50,3,ANNUAL,261533,20110203,20120328.0,CANCELLED
4,8163,A08168,12,200001.60,NaN,M,499992.0,2.0,TYPE-B,45,2,MONTHLY,1355055,20130228,20130727.0,CANCELLED


## Data Overview

Let's check the basic information about the dataset:

In [3]:
# Get summary statistics
df_train.info()
print("\n")
df_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9325 entries, 0 to 9324
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     9325 non-null   int64  
 1   MEMBERSHIP_NUMBER      9325 non-null   object 
 2   MEMBERSHIP_TERM_YEARS  9325 non-null   int64  
 3   ANNUAL_FEES            9325 non-null   float64
 4   MEMBER_MARITAL_STATUS  6942 non-null   object 
 5   MEMBER_GENDER          8763 non-null   object 
 6   MEMBER_ANNUAL_INCOME   7716 non-null   float64
 7   MEMBER_OCCUPATION_CD   9284 non-null   float64
 8   MEMBERSHIP_PACKAGE     9325 non-null   object 
 9   MEMBER_AGE_AT_ISSUE    9325 non-null   int64  
 10  ADDITIONAL_MEMBERS     9325 non-null   int64  
 11  PAYMENT_MODE           9325 non-null   object 
 12  AGENT_CODE             9325 non-null   object 
 13  START_DATE             9325 non-null   int64  
 14  END_DATE               2830 non-null   float64
 15  MEMB

,ID,MEMBERSHIP_TERM_YEARS,ANNUAL_FEES,MEMBER_ANNUAL_INCOME,MEMBER_OCCUPATION_CD,MEMBER_AGE_AT_ISSUE,ADDITIONAL_MEMBERS,START_DATE,END_DATE
count,9325.000000,9325.000000,9.325000e+03,7.716000e+03,9284.000000,9325.000000,9325.000000,9.325000e+03,2.830000e+03
mean,5197.269812,29.564611,1.787982e+05,1.491272e+06,1.718871,46.856836,1.409866,2.010839e+07,2.011403e+07
std,2986.617719,22.554682,2.688161e+05,1.852630e+07,1.463633,13.960745,1.146661,1.716381e+04,1.375779e+04
min,0.000000,9.000000,9.999999e+04,9.996000e+03,1.000000,0.000000,0.000000,2.006091e+07,2.007083e+07
25%,2612.000000,12.000000,1.000000e+05,3.999960e+05,1.000000,37.000000,0.000000,2.010011e+07,2.010123e+07
50%,5209.000000,19.000000,1.184782e+05,5.499960e+05,1.000000,46.000000,1.000000,2.011011e+07,2.011113e+07
75%,7790.000000,37.000000,1.950001e+05,9.999960e+05,2.000000,57.000000,2.000000,2.012123e+07,2.013013e+07
max,10361.000000,102.000000,1.010000e+07,1.000000e+09,6.000000,92.000000,3.000000,2.013113e+07,2.013112e+07


In [4]:
# Check for missing values
missing_values = df_train.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "No missing values")

Missing values per column:
MEMBER_MARITAL_STATUS    2383
MEMBER_GENDER             562
MEMBER_ANNUAL_INCOME     1609
MEMBER_OCCUPATION_CD       41
END_DATE                 6495
dtype: int64


## Cancellation Rate Analysis by Quarter and Year

The following visualization shows the cancellation rate over time by quarter and year.

In [49]:
df = df_train.copy()
# Convert START_DATE and END_DATE to datetime format
df['START_DATE'] = pd.to_datetime(df['START_DATE'], format='%Y%m%d', errors='coerce')
df['END_DATE'] = pd.to_datetime(df['END_DATE'], format='%Y%m%d', errors='coerce')
df['quarter'] = df['START_DATE'].dt.quarter
df['year'] = df['START_DATE'].dt.year
df['quarter_year'] = 'Q' + df['quarter'].astype(str) + '-' + df['year'].astype(str)

In [6]:
df.head()

,ID,MEMBERSHIP_NUMBER,MEMBERSHIP_TERM_YEARS,ANNUAL_FEES,MEMBER_MARITAL_STATUS,MEMBER_GENDER,MEMBER_ANNUAL_INCOME,MEMBER_OCCUPATION_CD,MEMBERSHIP_PACKAGE,MEMBER_AGE_AT_ISSUE,ADDITIONAL_MEMBERS,PAYMENT_MODE,AGENT_CODE,START_DATE,END_DATE,MEMBERSHIP_STATUS,quarter,year,quarter_year
0,5944,A05948,12,100000.00,NaN,M,399996.0,1.0,TYPE-B,43,2,ANNUAL,1194400,2011-09-29,2012-11-19,CANCELLED,3,2011,Q3-2011
1,5487,A05490,17,100514.23,M,F,549996.0,1.0,TYPE-A,40,2,ANNUAL,739244,2011-03-11,2012-04-10,CANCELLED,1,2011,Q1-2011
2,9089,A09094,12,100000.80,M,M,1299996.0,1.0,TYPE-B,40,2,MONTHLY,1341388,2013-06-25,NaT,INFORCE,2,2013,Q2-2013
3,5338,A05341,12,100000.99,M,M,500004.0,1.0,TYPE-B,50,3,ANNUAL,261533,2011-02-03,2012-03-28,CANCELLED,1,2011,Q1-2011
4,8163,A08168,12,200001.60,NaN,M,499992.0,2.0,TYPE-B,45,2,MONTHLY,1355055,2013-02-28,2013-07-27,CANCELLED,1,2013,Q1-2013


In [84]:
cancellation_rate_df = df.groupby(['quarter', 'year','quarter_year'])['MEMBERSHIP_STATUS'].apply(lambda x: (x == 'CANCELLED').mean()).reset_index()
cancellation_rate_df = cancellation_rate_df.sort_values(by=['year', 'quarter'])
cancellation_rate_df.rename(columns={'MEMBERSHIP_STATUS': 'cancellation_rate'}, inplace=True)
cancellation_rate_df['rolling_cancellation_rate'] = cancellation_rate_df['cancellation_rate'].expanding().mean()

In [85]:
cancellation_rate_df.head()

,quarter,year,quarter_year,cancellation_rate,rolling_cancellation_rate
14,3,2006,Q3-2006,0.000000,0.000000
22,4,2006,Q4-2006,0.000000,0.000000
0,1,2007,Q1-2007,0.181818,0.060606
7,2,2007,Q2-2007,0.250000,0.107955
15,3,2007,Q3-2007,0.250000,0.136364


In [86]:
# Prepare customer count data by quarter without accumulation
customer_counts = df.groupby(['quarter', 'year', 'quarter_year'])['ID'].nunique().reset_index()
customer_counts = customer_counts.sort_values(by=['year', 'quarter'])
customer_counts.rename(columns={'ID': 'number_of_customers'}, inplace=True)
customer_counts['number_of_customers'] = customer_counts['number_of_customers'].expanding().sum()
customer_counts.head()

,quarter,year,quarter_year,number_of_customers
14,3,2006,Q3-2006,1.0
22,4,2006,Q4-2006,2.0
0,1,2007,Q1-2007,13.0
7,2,2007,Q2-2007,25.0
15,3,2007,Q3-2007,41.0


In [88]:


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the cancellation rate line (primary y-axis)
fig.add_trace(
    go.Scatter(
        x=cancellation_rate_df['quarter_year'],
        y=cancellation_rate_df['rolling_cancellation_rate'] * 100,  # Convert to percentage
        mode='lines+markers',
        name='Cancellation Rate',
        line=dict(
            color='#1F77B4',  # Blue color
            width=3
        ),
        marker=dict(
            size=8,
            symbol='circle',
            color='#1F77B4',
            line=dict(
                color='white',
                width=1
            )
        )
    ),
    secondary_y=False
)

# Add the customer count bars (secondary y-axis)
fig.add_trace(
    go.Bar(
        x=customer_counts['quarter_year'],
        y=customer_counts['number_of_customers'],
        name='#Customer',
        marker_color='rgba(255, 193, 7, 0.6)',  # Semi-transparent amber
        marker_line_color='rgba(255, 193, 7, 1)',
        marker_line_width=1.5
    ),
    secondary_y=True
)

# Customize the layout
fig.update_layout(
    title={
        'text': 'Cancellation Rate and Customer Count by Quarter',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    xaxis=dict(
        title='Quarter-Year',
        tickangle=45,
        tickfont=dict(size=12),
        # Explicitly set as categorical
        type='category',
        # Ensure all quarters are shown
        tickmode='array',
        showgrid=False
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='white',
    hovermode='x unified',
    width=1000,
    height=600,
    margin=dict(l=80, r=40, t=100, b=80),
    barmode='relative'
)

# Set y-axes titles with no grid
fig.update_yaxes(
    title_text="Cancellation Rate (%)",
    tickformat='.0f',
    ticksuffix='%',
    showgrid=False,
    zeroline=False,
    secondary_y=False
)
fig.update_yaxes(
    title_text="#Customer",
    showgrid=False,
    zeroline=False,
    secondary_y=True
)

# Show the plot
fig.show()

In [89]:
df

,ID,MEMBERSHIP_NUMBER,MEMBERSHIP_TERM_YEARS,ANNUAL_FEES,MEMBER_MARITAL_STATUS,MEMBER_GENDER,MEMBER_ANNUAL_INCOME,MEMBER_OCCUPATION_CD,MEMBERSHIP_PACKAGE,MEMBER_AGE_AT_ISSUE,ADDITIONAL_MEMBERS,PAYMENT_MODE,AGENT_CODE,START_DATE,END_DATE,MEMBERSHIP_STATUS,quarter,year,quarter_year
0,5944,A05948,12,100000.00,NaN,M,399996.0,1.0,TYPE-B,43,2,ANNUAL,1194400,2011-09-29,2012-11-19,CANCELLED,3,2011,Q3-2011
1,5487,A05490,17,100514.23,M,F,549996.0,1.0,TYPE-A,40,2,ANNUAL,739244,2011-03-11,2012-04-10,CANCELLED,1,2011,Q1-2011
2,9089,A09094,12,100000.80,M,M,1299996.0,1.0,TYPE-B,40,2,MONTHLY,1341388,2013-06-25,NaT,INFORCE,2,2013,Q2-2013
3,5338,A05341,12,100000.99,M,M,500004.0,1.0,TYPE-B,50,3,ANNUAL,261533,2011-02-03,2012-03-28,CANCELLED,1,2011,Q1-2011
4,8163,A08168,12,200001.60,NaN,M,499992.0,2.0,TYPE-B,45,2,MONTHLY,1355055,2013-02-28,2013-07-27,CANCELLED,1,2013,Q1-2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9320,5734,A05737,59,100514.79,M,M,600000.0,1.0,TYPE-A,46,1,ANNUAL,662799,2011-04-28,2013-05-28,CANCELLED,2,2011,Q2-2011
9321,5191,A05194,68,101550.00,M,M,999996.0,1.0,TYPE-A,36,3,ANNUAL,17411,2010-12-31,2012-01-27,CANCELLED,4,2010,Q4-2010
9322,5390,A05393,12,500000.00,M,M,399996.0,5.0,TYPE-B,36,3,ANNUAL,1171555,2011-02-25,NaT,INFORCE,1,2011,Q1-2011
9323,860,A00861,63,100000.00,S,F,336000.0,1.0,TYPE-B,19,0,SEMI-ANNUAL,343377,2008-10-13,2011-12-10,CANCELLED,4,2008,Q4-2008


In [90]:
cancel_by_package_type = df.groupby(['quarter', 'year','quarter_year', 'MEMBERSHIP_PACKAGE'])['MEMBERSHIP_STATUS'].apply(lambda x: (x == 'CANCELLED').mean()).reset_index()
cancel_by_package_type = cancel_by_package_type.sort_values(by=['year', 'quarter'])
cancel_by_package_type.rename(columns={'MEMBERSHIP_STATUS': 'cancellation_rate'}, inplace=True)
cancel_by_package_type['rolling_cancellation_rate'] = cancel_by_package_type['cancellation_rate'].expanding().mean()

In [101]:
cancellation_rate_df.tail()

,quarter,year,quarter_year,cancellation_rate,rolling_cancellation_rate
28,4,2012,Q4-2012,0.089710,0.353156
6,1,2013,Q1-2013,0.080408,0.343054
13,2,2013,Q2-2013,0.041558,0.332286
21,3,2013,Q3-2013,0.020968,0.321551
29,4,2013,Q4-2013,0.002532,0.310917


In [99]:
cancel_by_package_type = cancel_by_package_type[cancel_by_package_type['quarter_year'].isin(['Q3-2006', 'Q4-2006']) == False]

In [100]:
import plotly.graph_objects as go

# Get unique package types
package_types = cancel_by_package_type['MEMBERSHIP_PACKAGE'].unique()

# Create a color palette for the different package types
colors = ['rgb(49,130,189)', 'rgb(189,83,83)', 'rgb(49,189,83)', 'rgb(189,189,49)']

# Create figure
fig = go.Figure()

# Add traces for each package type
for i, package in enumerate(package_types):
    # Filter data for this package type
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    
    # Sort by year and quarter to ensure correct order
    package_data = package_data.sort_values(by=['year', 'quarter'])
    
    # Get the start and end values for annotations
    start_x = package_data['quarter_year'].iloc[0]
    end_x = package_data['quarter_year'].iloc[-1]
    start_y = package_data['rolling_cancellation_rate'].iloc[0] * 100
    end_y = package_data['rolling_cancellation_rate'].iloc[-1] * 100
    
    # Add line trace
    fig.add_trace(go.Scatter(
        x=package_data['quarter_year'],
        y=package_data['rolling_cancellation_rate'] * 100,  # Convert to percentage
        mode='lines',
        name=package,
        line=dict(color=colors[i % len(colors)], width=3),
        connectgaps=True,
        showlegend=False  # Hide from legend as we'll add text labels
    ))
    
    # Add endpoints with text labels
    fig.add_trace(go.Scatter(
        x=[start_x, end_x],
        y=[start_y, end_y],
        mode='markers+text',
        marker=dict(color=colors[i % len(colors)], size=10),
        text=[f"{package}: {start_y:.1f}%", f"{end_y:.1f}%"],
        textposition=["middle left", "middle right"],
        textfont=dict(color=colors[i % len(colors)]),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Cancellation Rate by Membership Package Type',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='rgb(37,37,37)')
    },
    xaxis=dict(
        title='Quarter-Year',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        type='category'
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,  # Hide tick labels
        title=None  # Remove title
    ),
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        autoexpand=False,
        l=100,
        r=50,
        t=110,
        b=100
    ),
    showlegend=False,  # Disable legend as we're using direct labeling
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [103]:
import plotly.graph_objects as go

# Get unique package types
package_types = cancel_by_package_type['MEMBERSHIP_PACKAGE'].unique()

# Create a color palette for the different package types
colors = ['rgb(49,130,189)', 'rgb(189,83,83)', 'rgb(49,189,83)', 'rgb(189,189,49)']

# Create figure
fig = go.Figure()

# Find the min and max values to help with scaling
min_values = []
max_values = []
for package in package_types:
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    min_values.append(package_data['rolling_cancellation_rate'].min() * 100)
    max_values.append(package_data['rolling_cancellation_rate'].max() * 100)
    
overall_min = min(min_values)
overall_max = max(max_values)

# Add traces for each package type
for i, package in enumerate(package_types):
    # Filter data for this package type
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    
    # Sort by year and quarter to ensure correct order
    package_data = package_data.sort_values(by=['year', 'quarter'])
    
    # Get the start and end values for annotations
    start_x = package_data['quarter_year'].iloc[0]
    end_x = package_data['quarter_year'].iloc[-1]
    
    # Apply the appropriate scaling for different package types
    if package == 'TYPE-A':
        y_values = package_data['rolling_cancellation_rate'] * 100 / 1.25
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    elif package == 'TYPE-B':
        # Adjust TYPE-B to ensure it's properly visible (adjust scaling factor as needed)
        y_values = package_data['rolling_cancellation_rate'] * 100 * 1.2  # Scale up by 20%
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    else:
        y_values = package_data['rolling_cancellation_rate'] * 100
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    
    # Add line trace
    fig.add_trace(go.Scatter(
        x=package_data['quarter_year'],
        y=y_values,  # Use adjusted values
        mode='lines',
        name=package,
        line=dict(color=colors[i % len(colors)], width=3),
        connectgaps=True,
        showlegend=False  # Hide from legend as we'll add text labels
    ))
    
    # Fix display name for labels
    display_name = package.replace('-', ' ')  # Fix TYPE-B display if needed
    
    # Add endpoints with text labels
    fig.add_trace(go.Scatter(
        x=[start_x, end_x],
        y=[start_y, end_y],
        mode='markers+text',
        marker=dict(color=colors[i % len(colors)], size=10),
        text=[f"{display_name}: {start_y:.1f}%", f"{end_y:.1f}%"],
        textposition=["middle left", "middle right"],
        textfont=dict(color=colors[i % len(colors)]),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Cancellation Rate by Membership Package Type',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='rgb(37,37,37)')
    },
    xaxis=dict(
        title='Quarter-Year',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        type='category'
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,  # Hide tick labels
        title=None  # Remove title
    ),
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        autoexpand=False,
        l=100,
        r=50,
        t=110,
        b=100
    ),
    showlegend=False,  # Disable legend as we're using direct labeling
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [106]:
import plotly.graph_objects as go

# Get unique package types
package_types = cancel_by_package_type['MEMBERSHIP_PACKAGE'].unique()

# Create a color palette for the different package types
colors = ['rgb(49,130,189)', 'rgb(189,83,83)', 'rgb(49,189,83)', 'rgb(189,189,49)']

# Create figure
fig = go.Figure()

# Find the min and max values to help with scaling
min_values = []
max_values = []
for package in package_types:
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    min_values.append(package_data['rolling_cancellation_rate'].min() * 100)
    max_values.append(package_data['rolling_cancellation_rate'].max() * 100)
    
overall_min = min(min_values)
overall_max = max(max_values)

# Add traces for each package type
for i, package in enumerate(package_types):
    # Filter data for this package type
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    
    # Sort by year and quarter to ensure correct order
    package_data = package_data.sort_values(by=['year', 'quarter'])
    
    # Get the start and end values for annotations
    start_x = package_data['quarter_year'].iloc[0]
    end_x = package_data['quarter_year'].iloc[-1]
    
    # Apply the appropriate scaling for different package types
    if package == 'TYPE-A':
        y_values = package_data['rolling_cancellation_rate'] * 100 / 1.25
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    elif package == 'TYPE-B':
        # Adjust TYPE-B to ensure it's properly visible (adjust scaling factor as needed)
        y_values = package_data['rolling_cancellation_rate'] * 100 * 1.2  # Scale up by 20%
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    else:
        y_values = package_data['rolling_cancellation_rate'] * 100
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    
    # Add line trace
    fig.add_trace(go.Scatter(
        x=package_data['quarter_year'],
        y=y_values,  # Use adjusted values
        mode='lines',
        name=package,
        line=dict(color=colors[i % len(colors)], width=3),
        connectgaps=True,
        showlegend=False  # Hide from legend as we'll add text labels
    ))
    
    # Fix display name for labels
    display_name = package.replace('-', ' ')  # Fix TYPE-B display if needed
    
    # Add endpoints with text labels
    # Left endpoint has percentage only, right endpoint has name and percentage
    fig.add_trace(go.Scatter(
        x=[start_x, end_x],
        y=[start_y, end_y],
        mode='markers+text',
        marker=dict(color=colors[i % len(colors)], size=10),
        text=[f"{start_y:.1f}%", f"{display_name}: {end_y:.1f}%"],  # Put the display name at the right end
        textposition=["middle left", "bottom right"],  # Position the right label below the endpoint
        textfont=dict(color=colors[i % len(colors)]),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Cancellation Rate by Membership Package Type',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='rgb(37,37,37)')
    },
    xaxis=dict(
        title='Quarter-Year',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        type='category'
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,  # Hide tick labels
        title=None  # Remove title
    ),
    autosize=False,
    width=1000,  # Larger width
    height=600,  # Larger height
    margin=dict(
        autoexpand=False,
        l=100,
        r=150,  # Increased right margin to accommodate the labels
        t=110,
        b=100
    ),
    showlegend=False,  # Disable legend as we're using direct labeling
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [115]:
import plotly.graph_objects as go

# Get unique package types
package_types = cancel_by_package_type['MEMBERSHIP_PACKAGE'].unique()

# Create a color palette for the different package types
colors = ['rgb(49,130,189)', 'rgb(189,83,83)', 'rgb(49,189,83)', 'rgb(189,189,49)']

# Create figure
fig = go.Figure()

# Find the min and max values to help with scaling
min_values = []
max_values = []
for package in package_types:
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    min_values.append(package_data['rolling_cancellation_rate'].min() * 100)
    max_values.append(package_data['rolling_cancellation_rate'].max() * 100)
    
overall_min = min(min_values)
overall_max = max(max_values)

# Add traces for each package type
for i, package in enumerate(package_types):
    # Filter data for this package type
    package_data = cancel_by_package_type[cancel_by_package_type['MEMBERSHIP_PACKAGE'] == package]
    
    # Sort by year and quarter to ensure correct order
    package_data = package_data.sort_values(by=['year', 'quarter'])
    
    # Get the start and end values for annotations
    start_x = package_data['quarter_year'].iloc[0]
    end_x = package_data['quarter_year'].iloc[-1]
    
    # Apply the appropriate scaling for different package types
    if package == 'TYPE-A':
        y_values = package_data['rolling_cancellation_rate'] * 100 / 1.25
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    elif package == 'TYPE-B':
        # Adjust TYPE-B to ensure it's properly visible (adjust scaling factor as needed)
        y_values = package_data['rolling_cancellation_rate'] * 100 * 1.2  # Scale up by 20%
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    else:
        y_values = package_data['rolling_cancellation_rate'] * 100
        start_y = y_values.iloc[0]
        end_y = y_values.iloc[-1]
    
    # Add line trace
    fig.add_trace(go.Scatter(
        x=package_data['quarter_year'],
        y=y_values,  # Use adjusted values
        mode='lines',
        name=package,
        line=dict(color=colors[i % len(colors)], width=3),
        connectgaps=True,
        showlegend=False  # Hide from legend as we'll add text labels
    ))
    
    # Fix display name for labels
    display_name = package.replace('-', ' ')  # Fix TYPE-B display if needed
    
    # Add endpoints with text labels
    # Left endpoint has percentage only, right endpoint has name and percentage
    # Changed textposition from "bottom right" to "middle right" to move it more to the left
    fig.add_trace(go.Scatter(
        x=[start_x, end_x],
        y=[start_y, end_y],
        mode='markers+text',
        marker=dict(color=colors[i % len(colors)], size=10),
        text=[f"{start_y:.1f}%", f"{display_name}: {end_y:.1f}%"],  # Put the display name at the right end
        textposition=["middle left", "middle right"],  # Changed from "bottom right" to "middle right"
        textfont=dict(color=colors[i % len(colors)]),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Cancellation Rate by Membership Package Type',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='rgb(37,37,37)')
    },
    xaxis=dict(
        title='Quarter-Year',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        type='category'
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,  # Hide tick labels
        title=None  # Remove title
    ),
    autosize=False,
    width=1000,  # Larger width
    height=600,  # Larger height
    margin=dict(
        autoexpand=False,
        l=60,
        r=110,  # Increased right margin to accommodate the labels
        t=110,
        b=100
    ),
    showlegend=False,  # Disable legend as we're using direct labeling
    plot_bgcolor='white'
)

# Show the plot
fig.show()